In [21]:
#Scan through stock related tweets and derive sentiments by each user.
import sys
import os
import json
import string
import pprint
import csv
import re
from datetime import datetime
import math
import numpy as np

company_list = []
frequency = {}
with open('SP500.csv') as SP500csv:
    reader = csv.DictReader(SP500csv)
    for row in reader:
        company_list.append(row['Symbol'])

buy_keywords = []
sell_keywords = []
with open('keyword.csv') as keyword:
    reader = csv.DictReader(keyword)
    for row in reader:
        buy_keywords.append(row['Buy'])
        if row['Sell']:
            sell_keywords.append(row['Sell'])
        
#Create sentiment accumulation for different dates
day = 30
n_company = len(company_list)
acc_sentiment = [0] * n_company
for i in range(n_company):
    acc_sentiment[i] = [0] * day
    
arr = np.array(acc_sentiment)

#Initialize acc_sentiment files.
path = os.path.join("Extra_Storage", "ArchiveteamTest", "2015", "11")
with open(os.path.join(path, "id_list.txt" )) as id_list:
    reader = id_list.readlines()
    idList = [x.strip() for x in reader]
    n_authors = len(idList)
    for user in idList:
        #print(arr)
        np.savetxt(os.path.join(path, 'User_Sentiment', 'acc_sentiment_2015_11_' + user + '.csv'), arr, fmt='%i', delimiter=',')
    

In [22]:
for dirPath, dirNames, fileNames in os.walk(path):
    for file in fileNames:
        if file.endswith("json"):
            filepath = os.path.join(dirPath, file)
            with open(filepath) as json_data:
                data=[]
                for line in json_data:
                    try:
                        data.append(json.loads(line))
                    except json.JSONDecodeError:
                        continue
                for tweet in data:
                    if 'text' in tweet:
                        tweet_lower = tweet['text'].lower()
                        company_index = -1
                        for company in company_list:
                            company_index+=1
                            company_tag = ' $' + company + ' '
                            if company_tag.lower() in tweet_lower:
                                sentiment = 0
                                for buy_keyword in buy_keywords:
                                    if buy_keyword in tweet_lower:
                                        sentiment+=1
                                for sell_keyword in sell_keywords:
                                    if sell_keyword in tweet_lower:
                                        sentiment-=1
                                if sentiment==0:
                                    break
                                else:
                                    if 'created_at' in tweet:
                                        second = (datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +%f %Y')- datetime(2015, 11, 1)).total_seconds()
                                        date = math.ceil(second/86400) - 1            
                                    else:
                                        break
                                        
                                    user_sentiment_file = os.path.join(path, "User_Sentiment", "acc_sentiment_2015_11_" + str(tweet['user']['id']) + ".csv")
                                    user_sentiment = np.genfromtxt(user_sentiment_file, delimiter=',')
                                    if date ==  user_sentiment.shape[1]:
                                        date -= 1
                                    if sentiment > 0:
                                        user_sentiment[company_index][date]+=1
                                    else:
                                        user_sentiment[company_index][date]-=1
                                    #print(user_sentiment)        
                                    arr = np.array(user_sentiment)
                                    #np.savetxt(user_sentiment_file, arr, fmt='%i', header=dates, comments='', delimiter=',')
                                    np.savetxt(user_sentiment_file, arr, fmt='%i', delimiter=',')

In [23]:
#Generate dates
dates = []
for thedate in range(1, day+1):
    dates.append('11/'+ str(thedate) + '/2015')
    #dates += '4/' + str(thedate) + '/2016, '
    #dates += '4/30/2016' #Add final day

sentimentpath = os.path.join("Extra_Storage", "ArchiveteamTest", "2015", "11", "User_Sentiment")
for dirPath, dirNames, fileNames in os.walk(sentimentpath):
    for file in fileNames:
        filepath = os.path.join(dirPath, file)
        with open(filepath) as outcsv:
            r = csv.reader(outcsv)
            data = [line for line in r]
        with open(filepath, 'w', newline='') as outcsv:
            writer = csv.writer(outcsv)
            writer.writerow(dates)
            writer.writerows(data)